In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR
import re
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from transformers import BitsAndBytesConfig
import numpy as np

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load a pre-trained model

## Load from HuggingFace

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)

tokenizer.save_pretrained("tinyllama")
model.save_pretrained("tinyllama")

## Load from local repo

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(
    "./tinyllama"
)
base = AutoModelForCausalLM.from_pretrained(
    "./tinyllama",
    torch_dtype=torch.float16
).to(device)

In [3]:
prompt = "Il était tard lorsque  K. arriva. Une neige épaisse couvrait le village. La colline était cachée par la brume et par la nuit, nul rayon de lumière n’indiquait le grand Château."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

base.eval()

with torch.no_grad():
    output = base.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.8,
        top_p=0.9,
        do_sample=True,
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))

Il était tard lorsque  K. arriva. Une neige épaisse couvrait le village. La colline était cachée par la brume et par la nuit, nul rayon de lumière n’indiquait le grand Château.

Et, lorsque l’inspecteur arriva, tout était bien. Les portes et les fenêtres de la maison étaient fermées. Les volets et les fenêtres étaient fermés. Le chauffage et la lumière étaient désactivés. Les pattes de loup, avec leurs gants d’arme, étalées sur la table, ne faisaient aucune lumière. L’inspecteur K. s’assit à la table et regarda le visage du chien, avec une expression de désespoir.

L’ins


## Prepare the dataset

In [ ]:
with open("chateau.txt", "r", encoding="utf-8") as f:
    txt = f.read()

# remove page numbers
txt = re.sub(r"–\s*\d+\s*–\n", "", txt)

# fix split words
txt = re.sub(r"-\n", "", txt)

# remove line breaks
txt = re.sub(r"\n", " ", txt)

# use a single type of -
txt = re.sub(r"–", "-", txt)

with open("clean.txt", "w", encoding="utf-8") as f:
    f.write(txt)

In [4]:
txt = open("clean.txt", encoding="utf-8").read()
tokens = tokenizer.encode(txt)

Token indices sequence length is longer than the specified maximum sequence length for this model (210396 > 2048). Running this sequence through the model will result in indexing errors


## Fine-tune on Le Château by Kafka

### Prepare LoRA

In [5]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(base, lora_config)
model.print_trainable_parameters()

trainable params: 4,505,600 || all params: 1,104,553,984 || trainable%: 0.4079


### train

In [6]:
seq_len = 128
stride = seq_len // 2

inputs, targets = [], []
for i in range(0, len(tokens) - seq_len - 1, stride):
    chunk = tokens[i:i + seq_len + 1]
    inputs.append(chunk[:-1])
    targets.append(chunk[1:])

X = torch.tensor(inputs, dtype=torch.long)
Y = torch.tensor(targets, dtype=torch.long)

# split : premier batch pour validation
val_X, val_Y = X[:1], Y[:1]
train_X, train_Y = X[1:], Y[1:]

train_loader = DataLoader(TensorDataset(train_X, train_Y), batch_size=2, shuffle=True)
val_loader = DataLoader(TensorDataset(val_X, val_Y), batch_size=1, shuffle=False)

optimizer = AdamW(model.parameters(), lr=1e-5)
total_steps = len(train_loader)
print(total_steps)
warmup_steps = int(total_steps * 0.05)
scheduler = SequentialLR(
    optimizer,
    schedulers=[
        LinearLR(optimizer, start_factor=0.1, total_iters=warmup_steps),
        CosineAnnealingLR(optimizer, T_max=total_steps - warmup_steps, eta_min=1e-6),
    ],
    milestones=[warmup_steps],
)

log_path = "tinyllama_kafka.log"
with open(log_path, "w", encoding="utf-8") as log:
    log.write("step,loss,lr\n")

model.train()
total_loss = 0

for step, (Xb, Yb) in enumerate(train_loader, start=1): # keep the first one for validation
    Xb, Yb = Xb.to(device), Yb.to(device)
    outputs = model(Xb, labels=Yb)
    loss = outputs.loss
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    total_loss += loss.item()
    avg_loss = total_loss / step
    lr_now = scheduler.get_last_lr()[0]

    if step % 10 == 0:
        with open(log_path, "a", encoding="utf-8") as log:
            log.write(f"{step},{avg_loss:.6f},{lr_now:.2e}\n")
            
model.save_pretrained("tinyllama_kafka")

1643


## convert to weights

In [ ]:
base = AutoModelForCausalLM.from_pretrained(
    "tinyllama",
    torch_dtype=torch.float16
)
model = PeftModel.from_pretrained(
    base,
    "tinyllama_kafka",
    torch_dtype=torch.float16
)
model = model.merge_and_unload()
torch.save(model.state_dict(), "tinyllama_kafka.pt")

## load the fine-tuned model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(
    "./tinyllama"
)
model = AutoModelForCausalLM.from_pretrained(
    "./tinyllama",
    torch_dtype=torch.float16
).to(device)

model.load_state_dict(torch.load("tinyllama_kafka.pt", map_location=device))

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "tinyllama_kafka",
    torch_dtype=torch.float16
).to(device)

In [8]:
prompt = "Il était tard lorsque  K. arriva. Une neige épaisse couvrait le village. La colline était cachée par la brume et par la nuit, nul rayon de lumière n’indiquait le grand Château."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

model.eval()

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=1000,
        temperature=0.8,
        top_p=0.9,
        do_sample=True,
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))

Il était tard lorsque  K. arriva. Une neige épaisse couvrait le village. La colline était cachée par la brume et par la nuit, nul rayon de lumière n’indiquait le grand Château. ilait le en len, il avait parment lai une, seil des et et les’aires, avaité quil lait’’’ le. ler’’ -’- Il’é à. - Il’, K -’’ K -’ - K - -, le - de la, - il sait que. K nea jamaiséc le,’ l-aé,ant leait’ le - son’,’’ -’’ -’ ililait’ à’ qui’ il était. le’, s’a l -, - ne a été n à ne pas - pas il - aé par’ en de. le,’ Ka à - l,. ce qui’’’ de. qu le avait été le- - ?’ -, l’ -’. il neait - qui le a pasé et en’é pour -.,, K,’’ - qui a été’., - K,’ - lait -’ -, qui -’,’ -. ce’ - je lui, -’. le ne’, -i dit -.’ -’ -, K, - vous dit - -, -’ -. - vous nez pas’, vousous - lût, lil’. vous,- vous -, - - n - - dezendre le - ; nousions à, je suis moi,’,,’ K,.,. lui la’. ne nest pas - moi’.’ K -, - le, sa - son - en,. - -’ -,’ -’ - -, K,’ -,’ --’ -, le - de’.’ - le -’ -’ -,’ K -, - je mais’, neais -’ de’ - - je -’ - -. je vois’, mo

## check

In [16]:
base = AutoModelForCausalLM.from_pretrained(
    "./tinyllama",
    torch_dtype=torch.float16
)

total_elems, diff_elems = 0, 0

with torch.no_grad():
    for (name, p1), (_, p2) in zip(base.state_dict().items(), model.state_dict().items()):
        a = p1.detach().cpu().float().numpy()
        b = p2.detach().cpu().float().numpy()
        total_elems += a.size
        diff_elems += np.sum(np.abs(a - b) > 1e-5)

ratio = 100 * diff_elems / total_elems
print(f"{ratio:.4f}% of weights have been fine-tuned")

16.6627% of weights have been fine-tuned
